In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [2]:
# Function to extract the product title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        return title.text.strip() if title else ""
    except AttributeError:
        return ""

# Function to extract the product price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'class': 'a-offscree'})
        return price.text.strip() if price else ""
    except AttributeError:
        return ""

# Function to extract the product rating
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={'class': 'a-icon-alt'})
        return rating.text.strip() if rating else ""
    except AttributeError:
        return ""

# Function to extract the number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'})
        return review_count.text.strip() if review_count else ""
    except AttributeError:
        return ""


In [3]:
if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    URL = "https://www.amazon.com/s?k=nintendo+games"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch product links
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = ["https://www.amazon.com" + link.get('href') for link in links]

    # Initialize a dictionary to hold product data
    product_data = {"title": [], "price": [], "rating": [], "reviews": []}

    # Loop through each link and extract product details
    for link in links_list:
        new_webpage = requests.get(link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        product_data['title'].append(get_title(new_soup))
        product_data['price'].append(get_price(new_soup))
        product_data['rating'].append(get_rating(new_soup))
        product_data['reviews'].append(get_review_count(new_soup))

        # Respectful scraping
        time.sleep(5)  # Wait for 2 seconds before the next request

    # Create a DataFrame and clean it
    amazon_df = pd.DataFrame.from_dict(product_data)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])

    # Save the DataFrame to a CSV file (optional)
    # amazon_df.to_csv("amazon_data.csv", header=True, index=False)

ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: /aax-us-iad.amazon.com/x/c/JHs0hwUiruKvcDc58PLMGE4AAAGSHS1CUAEAAAH2AQBvbm9fdHhuX2JpZDcgICBvbm9fdHhuX2ltcDEgICAb3Tcm/https://www.amazon.com/Gammeefy-Controller-Replacement-Controler%EF%BC%8CSwitch-Controllers/dp/B09QPWT86S/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602%3Aamzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602&cv_ct_cx=nintendo+games&dib=eyJ2IjoiMSJ9.p-bLC4bZRAiYqLVZoWIudQ.J9hKcOtbVu6gb0D4V-VTXg-qnEMMJnbARTpoDu3dZc0&dib_tag=se&keywords=nintendo+games&pd_rd_i=B09QPWT86S&pd_rd_r=81de4ab4-a1a0-460b-8eee-74984c2885d3&pd_rd_w=2UGeA&pd_rd_wg=5qYuO&pf_rd_p=2f0a8989-0b67-47e7-b61e-9e3ef9908602&pf_rd_r=AVKE5B172F8TVMQ4T7JH&qid=1727066358&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-1-a61ee601-6e56-4862-a8a2-1d3da5a5406f (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002683A91F800>: Failed to resolve 'www.amazon.com

In [4]:
amazon_df

NameError: name 'amazon_df' is not defined